In [2]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression

spark = SparkSession.builder.appName('ml-youtube').getOrCreate()
transco_df = spark.read.csv('hdfs:///user/hduser/videodatasrc/transco.tsv', header = True, inferSchema = True, sep='\t')
transco_df.cache()
transco_df.printSchema()
transco_df.describe().toPandas().transpose()

root
 |-- id: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- codec: string (nullable = true)
 |-- width: integer (nullable = true)
 |-- height: integer (nullable = true)
 |-- bitrate: integer (nullable = true)
 |-- framerate: double (nullable = true)
 |-- i: integer (nullable = true)
 |-- p: integer (nullable = true)
 |-- b: integer (nullable = true)
 |-- frames: integer (nullable = true)
 |-- i_size: integer (nullable = true)
 |-- p_size: integer (nullable = true)
 |-- b_size: integer (nullable = true)
 |-- size: integer (nullable = true)
 |-- o_codec: string (nullable = true)
 |-- o_bitrate: integer (nullable = true)
 |-- o_framerate: double (nullable = true)
 |-- o_width: integer (nullable = true)
 |-- o_height: integer (nullable = true)
 |-- umem: integer (nullable = true)
 |-- utime: double (nullable = true)



0                     1                     2            3  \
summary      count                  mean                stddev          min   
id           23952                  None                  None  04t6-jw9czg   
duration     23952     269.0939810371274    216.40984965324515       34.168   
codec        23952                  None                  None          flv   
width        23952     620.6813627254509     445.3066304019825          176   
height       23952    408.57715430861725    228.24755015866333          144   
bitrate      23952     706756.8366733467    1172269.1997811738        52783   
framerate    23952     23.51818095791549     6.923677731222053         12.0   
i            23952     83.96893787575151    53.292537596502065           13   
p            23952     5924.861723446894     4770.190982167013          936   
b            23952                   0.0                   0.0            0   
frames       23952     6008.830661322645     4808.081979181758          975   
i_size       23952    1963229.0741482966     2992209.317117459        64483   
p_size       23952  1.7467511295591183E7  2.4345046863807943E7       559745   
b_size       23952                   0.0                   0.0            0   
size         23952   1.943074036973948E7   2.638629973059741E7       798230   
o_codec      23952                  None                  None          flv   
o_bitrate    23952    1394471.9438877755    1748795.0713637306        56000   
o_framerate  23952     21.19014529057857     6.668200591713311         12.0   
o_width      23952     802.6666666666666     610.0036264734396          176   
o_height     23952                 504.0    315.98127873453114          144   
umem         23952    207178.72578490313    105008.69574796125        22508   
utime        23952    10.187347820641238    16.339615317021696        0.268   

                       4  
summary              max  
id           1s9d-jTKyUw  
duration        906.5983  
codec                vp8  
width               1920  
height              1080  
bitrate          5992818  
framerate       30.38983  
i                    245  
p                  21264  
b                      0  
frames             21407  
i_size          14532530  
p_size         105950761  
b_size                 0  
size           115083398  
o_codec              vp8  
o_bitrate        5000000  
o_framerate        29.97  
o_width             1920  
o_height            1080  
umem              706156  
utime            214.481

In [4]:
import six
for i in transco_df.columns:
    if not( isinstance(transco_df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to utime for ", i, transco_df.stat.corr('utime',i))

Correlation to utime for  duration -0.03179669057693282
Correlation to utime for  width 0.12628287411698266
Correlation to utime for  height 0.12729911442100825
Correlation to utime for  bitrate 0.1466829557361457
Correlation to utime for  framerate 0.08164135724481011
Correlation to utime for  i -0.0278899278191425
Correlation to utime for  p 0.001966300084651395
Correlation to utime for  b nan
Correlation to utime for  frames 0.0016416737358781778
Correlation to utime for  i_size 0.041726405337875264
Correlation to utime for  p_size 0.06102864538652319
Correlation to utime for  b_size nan
Correlation to utime for  size 0.06103922820689375
Correlation to utime for  o_bitrate 0.162256760654313
Correlation to utime for  o_framerate 0.10388800675855171
Correlation to utime for  o_width 0.5271623208577183
Correlation to utime for  o_height 0.5235172916353216
Correlation to utime for  umem 0.6767112912323565
Correlation to utime for  utime 1.0


In [8]:
categoricalColumns = ['id','codec','o_codec']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
#StringIndexer encodes a string column of labels to a column of label indices.
#id | category | categoryIndex
#----|----------|---------------
# 0  | a        | 0.0
# 1  | b        | 2.0
# 2  | c        | 1.0
# 3  | a        | 0.0
# 4  | a        | 0.0
# 5  | c        | 1.0
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
# categorical data must be converted to a numerical form using onehot encoder
numericCols = ['width','height','bitrate','framerate','frames','i_size','p_size','size','o_bitrate','o_framerate','o_width','o_height','umem','utime']

assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
# VectorAssembler is used for assembling features into a vector. We will pass all the columns that we are going to use 
# for the prediction to the VectorAssembler and it will create a new vector column.
vectorAssembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [vectorAssembler]

pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(transco_df)
#When the PipelineModel’s transform() method is called on a test dataset, the data are passed through the fitted pipeline in order. 
#Each stage’s transform() method updates the dataset and passes it to the next stage.
df = pipelineModel.transform(transco_df)
df_featu = df.select(['features', 'utime', 'codec', 'frames'])
df_featu.show(3,False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-----+------+
|features                                                                                                                                                               |utime|codec|frames|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-----+------+
|(48,[8,30,31,34,35,36,37,38,39,40,41,42,43,44,45,46,47],[1.0,1.0,1.0,176.0,144.0,54590.0,12.0,1564.0,64483.0,825054.0,889537.0,56000.0,12.0,176.0,144.0,22508.0,0.612])|0.612|mpeg4|1564  |
|(48,[8,30,31,34,35,36,37,38,39,40,41,42,43,44,45,46,47],[1.0,1.0,1.0,176.0,144.0,54590.0,12.0,1564.0,64483.0,825054.0,889537.0,56000.0,12.0,320.0,240.0,25164.0,0.98]) |0.98 |mpeg4|1564  |
|(48,[8,30,31,34,35,36,37,38,39,40,41,42,43,44,45,46,47

In [10]:
splits = df_featu.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]	
print(" Training data count " + repr(train_df.count()))
print(" Testing data Count " + repr(test_df.count()))
train_df.show(3,False)
test_df.show(3,False)
train_df.describe().show()

 Training data count 16882
 Testing data Count 7070
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-----+------+
|features                                                                                                                                                                                |utime|codec|frames|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-----+------+
|(48,[0,29,31,34,35,36,37,38,39,40,41,42,43,44,45,46,47],[1.0,1.0,1.0,1280.0,720.0,794075.0,13.482667,10112.0,3852576.0,7.0635013E7,7.4487589E7,56000.0,12.0,320.0,240.0,165700.0,2.188])|2.188|vp8  |10112 |
|(48,[0,29,31,34,35,36,37,38,39,40,41,42,43,44,45,46,47],[1.0,1.0,1.0,1280.0,720.0,794075.0,13.482667,10112.0,3852576.0,7.06

In [11]:
#Linear regression is used for finding linear relationship between target and one or more predictors.
#The elastic net regulation method overcomes the limitations of the LASSO (least absolute shrinkage and selection operator) method which uses a penalty function
#α(λ∥w∥1)+(1−α)(λ2∥w∥22),α∈[0,1],λ≥0 (elasticNetParam corresponds to α and regParam corresponds to λ.)
lr = LinearRegression(featuresCol = 'features', labelCol='utime', maxIter=10, regParam=0.3, elasticNetParam=0.8 )
lr_model = lr.fit(train_df)
#Regression coefficients represent the mean change in the response variable for one unit of change in the 
#predictor variable while holding other predictors in the model constant.
print("Coefficients: " + str(lr_model.coefficients))
#The intercept (often labeled the constant) is the expected mean value of Y when all X=0. 
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9831043434361372]
Intercept: 0.17094604270994446


In [12]:
trainingSummary = lr_model.summary
#R-squared is a relative measure of fit, RMSE is an absolute measure of fit., Lower values of RMSE indicate better fit.
#R-squared has the useful property it ranges from zero to one, with zero indicating that the proposed model does not 
# improve prediction over the mean model, and one indicating perfect prediction
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 0.273836
r2: 0.999715


In [13]:
# When the PipelineModel’s transform() method is called on a test dataset, the data are passed through the fitted pipeline in order. Each stage’s transform() method updates the dataset and passes it to the next stage.
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","utime","codec", "frames").show(10,False)
lr_predictions.select("prediction","utime","codec", "frames").coalesce(1).write.csv("file:///home/hduser/mloutput",header = True, mode = "Overwrite")

+------------------+------+-----+------+
|prediction        |utime |codec|frames|
+------------------+------+-----+------+
|2.0467091299860942|1.908 |vp8  |10112 |
|7.103797872621584 |7.052 |vp8  |10112 |
|5.4757770798913405|5.396 |vp8  |10112 |
|4.3864974673641015|4.288 |vp8  |10112 |
|7.607147296460886 |7.564 |vp8  |10112 |
|11.501223600811427|11.525|vp8  |10112 |
|8.748531439190241 |8.725 |vp8  |10112 |
|12.146140050105531|12.181|vp8  |10112 |
|5.015684247163229 |4.928 |vp8  |10112 |
|4.052241990595814 |3.948 |vp8  |10112 |
+------------------+------+-----+------+
only showing top 10 rows



In [15]:
#Linear regression is a linear model, which means it works really nicely when the data has a linear shape. 
# But, when the data has a non-linear shape, then a linear model cannot capture the non-linear features.
# So in this case, you can use the decision trees, which do a better job at capturing the non-linearity in the data by 
# dividing the space into smaller sub-spaces depending on the questions asked.
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'utime')
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="utime", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 3.65269


In [16]:
#GBM is being used widely, many practitioners still treat it as complex black-box algorithm and just run the models 
# using pre-built libraries.
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'utime', maxIter=10)
gbt_model = gbt.fit(train_df)
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', 'utime', 'features').show(5)

+------------------+-----+--------------------+
|        prediction|utime|            features|
+------------------+-----+--------------------+
|1.7697223909523996|1.908|(48,[0,29,31,34,3...|
| 7.273350936219759|7.052|(48,[0,29,31,34,3...|
| 5.803139233543024|5.396|(48,[0,29,31,34,3...|
| 3.837082836514948|4.288|(48,[0,29,31,34,3...|
| 7.303307712642409|7.564|(48,[0,29,31,34,3...|
+------------------+-----+--------------------+
only showing top 5 rows



In [17]:
gbt_evaluator = RegressionEvaluator(
    labelCol="utime", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 2.47353
